In [1]:
# import
import sqlalchemy
from configparser import ConfigParser

In [2]:
# Loading config file
parser = ConfigParser()
_ = parser.read('nb.cfg')

# sql config
%config SqlMagic.displaycon = False
engine = sqlalchemy.create_engine(parser.get('video_games_db', 'conn_string'))

%load_ext sql
%sql $engine.url

# 1. The ten best-selling video games
<p><img src="images/video_game.jpg" alt="A Video Game Controller" width="400"></p>
<p>Photo by <a href="https://unsplash.com/@jagg">Jose Gil</a> on <a href="https://unsplash.com">Unsplash</a>.</p>
<p>Video games are big business: the global gaming market is projected to be worth more than $300 billion by 2027 according to <a href="https://www.mordorintelligence.com/industry-reports/global-gaming-market">Mordor Intelligence</a>. With so much money at stake, the major game publishers are hugely incentivized to create the next big hit. But are games getting better, or has the golden age of video games already passed?</p>
<p>In this project, we'll explore the top 19599 best-selling video games created between 1977 and 2020. We'll compare a dataset on game sales with critic and user reviews to determine whether or not video games have improved as the gaming market has grown.</p>
<p>The data set has been collected from <a href="https://www.kaggle.com/holmjason2/videogamedata">Kaggle</a>. </p>

<h3 id="game_sales"><code>game_sales</code></h3>
<table>
<thead>
<tr>
<th style="text-align:left;">column</th>
<th>type</th>
<th>meaning</th>
</tr>
</thead>
<tbody>
<tr>
<td style="text-align:left;"><code>game_id</code></td>
<td>serial</td>
<td>Id of the video game</td>
</tr>
<tr>
<td style="text-align:left;"><code>game</code></td>
<td>varchar</td>
<td>Name of the video game</td>
</tr>
<tr>
<td style="text-align:left;"><code>platform</code></td>
<td>varchar</td>
<td>Gaming platform</td>
</tr>
<tr>
<td style="text-align:left;"><code>publisher</code></td>
<td>varchar</td>
<td>Game publisher</td>
</tr>
<tr>
<td style="text-align:left;"><code>developer</code></td>
<td>varchar</td>
<td>Game developer</td>
</tr>
<tr>
<td style="text-align:left;"><code>games_sold</code></td>
<td>float</td>
<td>Number of copies sold (millions)</td>
</tr>
<tr>
<td style="text-align:left;"><code>year</code></td>
<td>int</td>
<td>Release year</td>
</tr>
</tbody>
</table>
<h3 id="reviews"><code>reviews</code></h3>
<table>
<thead>
<tr>
<th style="text-align:left;">column</th>
<th>type</th>
<th>meaning</th>
</tr>
</thead>
<tbody>
<tr>
<td style="text-align:left;"><code>game_id</code></td>
<td>serial</td>
<td>Id of the video game</td>
</tr>
<tr>
<td style="text-align:left;"><code>game</code></td>
<td>varchar</td>
<td>Name of the video game</td>
</tr>
<tr>
<td style="text-align:left;"><code>critic_score</code></td>
<td>float</td>
<td>Critic score according to Metacritic</td>
</tr>
<tr>
<td style="text-align:left;"><code>user_score</code></td>
<td>float</td>
<td>User score according to Metacritic</td>
</tr>
</tbody>
</table>
<p>Let's begin by looking at some of the top selling video games of all time!</p>

In [3]:
%%sql

SELECT * 
FROM game_sales
ORDER BY games_sold DESC
LIMIT 10;

10 rows affected.


game_id,game,platform,publisher,developer,games_sold,year
1,Wii Sports,Wii,Nintendo,Nintendo EAD,82.9,2006
2,Super Mario Bros.,NES,Nintendo,Nintendo EAD,40.24,1985
3,Counter-Strike: Global Offensive,PC,Valve,Valve Corporation,40.0,2012
4,Mario Kart Wii,Wii,Nintendo,Nintendo EAD,37.32,2008
5,PLAYERUNKNOWN'S BATTLEGROUNDS,PC,PUBG Corporation,PUBG Corporation,36.6,2017
6,Minecraft,PC,Mojang,Mojang AB,33.15,2010
7,Wii Sports Resort,Wii,Nintendo,Nintendo EAD,33.13,2009
8,Pokemon Red / Green / Blue Version,GB,Nintendo,Game Freak,31.38,1998
9,New Super Mario Bros.,DS,Nintendo,Nintendo EAD,30.8,2006
10,New Super Mario Bros. Wii,Wii,Nintendo,Nintendo EAD,30.3,2009


# 2. Missing review scores
<p>We'll have to use data from the <code>reviews</code> table to gain more insight on the best years for video games. </p>
<p>First, it's important to explore the limitations of our database. Let's determine how many games in the <code>game_sales</code> table are missing both <code>user_score</code> and <code>critic_score</code>.

In [4]:
%%sql

SELECT COUNT(*)
FROM game_sales AS gs
INNER JOIN reviews AS r
ON gs.game_id=r.game_id
WHERE r.critic_score IS NULL AND r.user_score IS NULL;

1 rows affected.


count
9611


# 3. Years that video game critics loved
<p>The missing reviews data is a good thing to keep in mind as we move on to evaluating results from more sophisticated queries. </p>
<p>There are lots of ways to measure the best years for video games! Let's start with what the critics think. </p>

In [5]:
%%sql

SELECT year, ROUND(AVG(r.critic_score)::NUMERIC, 2) AS avg_critic_score
FROM game_sales AS gs
INNER JOIN reviews AS r
USING(game_id)
WHERE r.critic_score IS NOT NULL
GROUP BY year
ORDER BY avg_critic_score DESC
LIMIT 10;

10 rows affected.


year,avg_critic_score
1984,9.50
1992,9.13
1982,9.00
1994,8.72
1990,8.63
1991,8.49
2020,8.26
1993,8.03
2019,7.88
1989,7.66


# 4. Not enough game?
<p>The range of great years according to critic reviews goes from 1982 until 2020.</p>
<p>But, some of those <code>avg_critic_score</code> values look like suspiciously round numbers for averages. Like the value for 1982 and 1984 looks especially fishy. Maybe there weren't a lot of video games in our dataset that were released in certain years. </p>
<p>Let's check it and update our query to find out if certain years have enough games that were released.</p>

In [6]:
%%sql

SELECT year, ROUND(AVG(r.critic_score)::NUMERIC, 2) AS avg_critic_score, COUNT(gs.game) AS num_games
FROM game_sales AS gs
INNER JOIN reviews AS r
USING(game_id)
WHERE r.critic_score IS NOT NULL
GROUP BY year
ORDER BY avg_critic_score DESC
LIMIT 10;

10 rows affected.


year,avg_critic_score,num_games
1984,9.50,1
1992,9.13,4
1982,9.00,1
1994,8.72,6
1990,8.63,6
1991,8.49,8
2020,8.26,9
1993,8.03,11
2019,7.88,37
1989,7.66,5


### Looks like our doubt was right. 1982 and 1984 had only 1 game. Lets filter the years which had at least 10 games released.

In [7]:
%%sql

SELECT year, ROUND(AVG(r.critic_score)::NUMERIC, 2) AS avg_critic_score, COUNT(gs.game) AS num_games
FROM game_sales AS gs
INNER JOIN reviews AS r
USING(game_id)
WHERE r.critic_score IS NOT NULL
GROUP BY year
HAVING COUNT(gs.game) >= 10
ORDER BY avg_critic_score DESC
LIMIT 10;

10 rows affected.


year,avg_critic_score,num_games
1993,8.03,11
2019,7.88,37
2014,7.54,264
2013,7.46,284
2015,7.41,279
2016,7.39,311
2017,7.37,377
2012,7.37,329
2018,7.34,380
2003,7.27,564


# 5. Years video game players loved
<p>Let's move on to looking at the opinions of another important group of people: players! To begin, let's create a query very similar to the one we used in Task Four, except this one will look at <code>user_score</code> averages by year rather than <code>critic_score</code> averages.</p>

In [8]:
%%sql

SELECT year, ROUND(AVG(r.user_score)::NUMERIC, 2) AS avg_user_score, COUNT(gs.game) AS num_games
FROM game_sales AS gs
INNER JOIN reviews AS r
USING(game_id)
WHERE r.user_score IS NOT NULL
GROUP BY year
HAVING COUNT(gs.game) >= 10
ORDER BY avg_user_score DESC
LIMIT 10;

10 rows affected.


year,avg_user_score,num_games
2010,8.78,22
2009,8.57,44
2007,8.57,26
2008,8.40,52
2006,8.26,16
2005,7.62,13
2012,7.07,153
2014,7.01,226
2013,6.98,138
2016,6.92,295


# 6. Years that both players and critics loved
<p>Alright, we've got a list of the top ten years according to both <strong>critic reviews</strong> and <strong>user reviews</strong>. Are there any years that showed up on both tables? If so, those years would certainly be excellent ones!</p>

In [9]:
%%sql

SELECT year FROM
(
    SELECT year, ROUND(AVG(r.critic_score)::NUMERIC, 2) AS avg_critic_score, COUNT(gs.game) AS num_games
    FROM game_sales AS gs
    INNER JOIN reviews AS r
    USING(game_id)
    WHERE r.critic_score IS NOT NULL
    GROUP BY year
    HAVING COUNT(gs.game) >= 10
) AS top_critic_years_more_than_ten_games
INTERSECT
SELECT year FROM 
(
    SELECT year, ROUND(AVG(r.user_score)::NUMERIC, 2) AS avg_user_score, COUNT(gs.game) AS num_games
    FROM game_sales AS gs
    INNER JOIN reviews AS r
    USING(game_id)
    WHERE r.user_score IS NOT NULL
    GROUP BY year
    HAVING COUNT(gs.game) >= 10
) AS top_user_years_more_than_ten_games
ORDER BY year
LIMIT 10;

10 rows affected.


year
2005
2006
2007
2008
2009
2010
2011
2012
2013
2014


# 7. Sales in the best video game years
<p>Looks like we've got the years that both users and critics agreed were in the top ten! There are many other ways of measuring what the best years for video games are, but let's stick with these years for now. We know that critics and players liked these years, but what about video game makers? Were sales good? Let's find out.</p>

In [10]:
%%sql

SELECT year, ROUND(SUM(games_sold)::NUMERIC, 2) AS total_games_sold
FROM game_sales
WHERE year IN 
(
    SELECT year FROM
    (
        SELECT year, ROUND(AVG(r.critic_score)::NUMERIC, 2) AS avg_critic_score, COUNT(gs.game) AS num_games
        FROM game_sales AS gs
        INNER JOIN reviews AS r
        USING(game_id)
        WHERE r.critic_score IS NOT NULL
        GROUP BY year
        HAVING COUNT(gs.game) >= 10
    ) AS top_critic_years_more_than_ten_games
    INTERSECT
    SELECT year FROM 
    (
        SELECT year, ROUND(AVG(r.user_score)::NUMERIC, 2) AS avg_user_score, COUNT(gs.game) AS num_games
        FROM game_sales AS gs
        INNER JOIN reviews AS r
        USING(game_id)
        WHERE r.user_score IS NOT NULL
        GROUP BY year
        HAVING COUNT(gs.game) >= 10
    ) AS top_user_years_more_than_ten_games
    ORDER BY year
)
GROUP BY year
ORDER BY total_games_sold DESC
LIMIT 10;

10 rows affected.


year,total_games_sold
2008,734.61
2009,694.38
2010,676.41
2007,634.62
2011,633.36
2006,520.05
2013,470.88
2005,456.29
2012,455.69
2014,454.16
